# 📊 Cluster Configuration Analysis: Clusters, Owners & Workspaces

This notebook provides comprehensive analysis of cluster configuration changes from the [dev_sandbox.billing_forecast.cluster_config_backup](#table) table, focused on:

* **🖥️ Cluster-Level Analysis** - Individual cluster changes and configurations
* **👤 Owner-Based Analysis** - Changes grouped by cluster owners
* **🏢 Workspace Analysis** - Distribution across workspaces

---

## 📈 Available Visualizations:

### 1. **Cluster-Level Change Analysis**
* Horizontal bar chart showing changes per cluster
* Color-coded by impact level (MAJOR=red, MODERATE=orange, MINOR=green)
* Hover details include owner, instance transitions, and change counts

### 2. **Owner-Based Cluster Analysis**
* Grouped bar chart comparing clusters managed vs instance changes by owner
* Extracted from cluster names (e.g., "Gunjan Bali's cluster" → Owner: "Gunjan Bali")
* Shows which owners have the most clusters and changes

### 3. **Workspace → Cluster Hierarchy (Sunburst)**
* Interactive sunburst chart showing workspace → cluster relationships
* Color-coded by change impact
* Click to drill down into specific workspaces and clusters

### 4. **Cluster Instance Type Transitions**
* Stacked bar chart showing before/after instance types
* Separate bars for driver and worker instances
* Visual representation of m7g.large → m6gd.large transitions

### 5. **Owner vs Cluster Changes Scatter**
* Scatter plot showing relationship between owners and change counts
* Bubble size represents number of changes
* Color-coded by impact level

### 6. **Detailed Cluster Comparison Table**
* Interactive table with all cluster details
* Color-coded rows by impact level
* Includes owner, workspace, and full before/after configuration

### 7. **Change Types by Owner**
* Stacked bar chart showing different change categories per owner
* 9 change types tracked (instance, policy, spark config, tags, etc.)
* Identifies which owners had which types of changes

### 8. **Workspace → Owner → Cluster Treemap**
* Hierarchical treemap visualization
* Three levels: Workspace → Owner → Cluster
* Size represents total changes, color represents impact

---

## 💡 Key Insights:

* **24 clusters** updated in **Integrated-Dev** workspace
* All changes classified as **MAJOR** impact
* Primary change: **Instance type migrations** (m7g → m6gd)
* **24 unique owners** identified from cluster names
* Updates performed on **Dec 12, 2025**

---

## 🎨 Dashboard Integration:

All visualizations are created with **Plotly** and are **fully interactive**:
* Hover for detailed information
* Click and drag to zoom
* Double-click to reset view
* Click legend items to show/hide traces

**Ready for Lakeview Dashboard**: Simply add these cells to your Lakeview dashboard for interactive monitoring.

---

## 📝 Note on Timeseries:

The current data contains a **single snapshot** (Dec 12, 2025). For true timeseries analysis showing trends over time, additional backup records from different dates would be needed. The visualizations are designed to accommodate historical data when available.

## 🚀 How to Add These Visualizations to Lakeview Dashboard

### Step 1: Create or Open a Lakeview Dashboard
1. Navigate to **Dashboards** in your Databricks workspace
2. Click **Create Dashboard** or open an existing one
3. Select **Lakeview** as the dashboard type

### Step 2: Add Notebook Cells to Dashboard
1. In the dashboard editor, click **Add** → **Visualization**
2. Select **From Notebook**
3. Choose this notebook
4. Select the specific cell(s) you want to add
5. The Plotly visualizations will render with full interactivity

### Step 3: Arrange and Customize
1. Drag and resize visualizations on the dashboard canvas
2. Add filters, parameters, or text widgets as needed
3. Set refresh schedules for automatic updates
4. Share with your team

---

## ✨ Visualization Features:

* **Interactive Hover**: Rich tooltips with detailed information
* **Zoom & Pan**: Click and drag to explore data
* **Legend Controls**: Click legend items to show/hide data series
* **Responsive Design**: Automatically adjusts to dashboard layout
* **Professional Styling**: Color-coded by impact, clean typography
* **Export Ready**: Download as PNG/SVG for presentations

---

## 🎯 Best Practices:

* **Combine Multiple Views**: Add 3-4 complementary visualizations per dashboard page
* **Use Filters**: Add workspace or owner filters for dynamic exploration
* **Set Refresh Schedules**: Keep dashboards up-to-date automatically
* **Add Context**: Include text widgets explaining key metrics
* **Monitor Trends**: As more backup data accumulates, timeseries patterns will emerge

In [0]:
import plotly.graph_objs as go
import plotly.offline as pyo
import re

# Query cluster-level changes with owner extraction
df = spark.sql("""
    SELECT 
      cluster_name,
      workspace_name,
      before_driver_instance,
      after_driver_instance,
      before_worker_instance,
      after_worker_instance,
      change_impact,
      total_changes_count,
      backup_timestamp,
      updated_by_user
    FROM dev_sandbox.billing_forecast.cluster_config_backup
    WHERE update_status = 'SUCCESS'
    ORDER BY cluster_name
""")

pdf = df.toPandas()

# Extract owner from cluster name (pattern: "Name's ..." or just use cluster name)
pdf['owner'] = pdf['cluster_name'].apply(lambda x: x.split("'s")[0] if "'s" in x else x.split()[0] if len(x.split()) > 0 else 'Unknown')

# Create a horizontal bar chart showing changes per cluster
fig = go.Figure()

# Color by change impact
color_map = {'MAJOR': '#E74C3C', 'MODERATE': '#F39C12', 'MINOR': '#27AE60'}
colors = [color_map.get(impact, '#95A5A6') for impact in pdf['change_impact']]

fig.add_trace(go.Bar(
    y=pdf['cluster_name'],
    x=pdf['total_changes_count'],
    orientation='h',
    marker=dict(
        color=colors,
        line=dict(color='white', width=1)
    ),
    text=pdf['total_changes_count'],
    textposition='outside',
    hovertemplate='<b>%{y}</b><br>' +
                  'Owner: ' + pdf['owner'] + '<br>' +
                  'Changes: %{x}<br>' +
                  'Impact: ' + pdf['change_impact'] + '<br>' +
                  'Driver: ' + pdf['before_driver_instance'] + ' → ' + pdf['after_driver_instance'] + '<br>' +
                  'Worker: ' + pdf['before_worker_instance'] + ' → ' + pdf['after_worker_instance'] + '<br>' +
                  '<extra></extra>',
    customdata=pdf[['owner', 'change_impact', 'before_driver_instance', 'after_driver_instance', 'before_worker_instance', 'after_worker_instance']].values
))

fig.update_layout(
    title={
        'text': '🖥️ Cluster-Level Change Analysis',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 24, 'color': '#2C3E50', 'family': 'Arial Black'}
    },
    xaxis={
        'title': 'Number of Changes',
        'showgrid': True,
        'gridcolor': '#E8E8E8',
        'tickfont': {'size': 12}
    },
    yaxis={
        'title': '',
        'tickfont': {'size': 9, 'family': 'Arial'},
        'showgrid': False
    },
    height=800,
    plot_bgcolor='white',
    paper_bgcolor='#F8F9FA',
    hovermode='closest',
    font={'family': 'Arial, sans-serif', 'size': 12, 'color': '#2C3E50'},
    margin=dict(l=400, r=50, t=80, b=50),
    showlegend=False
)

displayHTML(pyo.plot(fig, output_type='div', include_plotlyjs='cdn'))

In [0]:
import plotly.graph_objs as go
import plotly.offline as pyo
import re

# Query and extract owner information
df = spark.sql("""
    SELECT 
      cluster_name,
      workspace_name,
      change_impact,
      total_changes_count,
      instance_type_changed,
      policy_changed,
      spark_config_changed
    FROM dev_sandbox.billing_forecast.cluster_config_backup
    WHERE update_status = 'SUCCESS'
""")

pdf = df.toPandas()

# Extract owner from cluster name
pdf['owner'] = pdf['cluster_name'].apply(lambda x: x.split("'s")[0] if "'s" in x else x.split()[0] if len(x.split()) > 0 else 'Unknown')

# Aggregate by owner
owner_stats = pdf.groupby('owner').agg({
    'cluster_name': 'count',
    'total_changes_count': 'sum',
    'instance_type_changed': 'sum',
    'policy_changed': 'sum',
    'spark_config_changed': 'sum'
}).reset_index()

owner_stats.columns = ['owner', 'clusters_managed', 'total_changes', 'instance_changes', 'policy_changes', 'config_changes']
owner_stats = owner_stats.sort_values('total_changes', ascending=True)

# Create horizontal bar chart
fig = go.Figure()

fig.add_trace(go.Bar(
    y=owner_stats['owner'],
    x=owner_stats['clusters_managed'],
    name='Clusters Managed',
    orientation='h',
    marker=dict(color='#3498DB', line=dict(color='white', width=1)),
    hovertemplate='<b>%{y}</b><br>' +
                  'Clusters: %{x}<br>' +
                  '<extra></extra>'
))

fig.add_trace(go.Bar(
    y=owner_stats['owner'],
    x=owner_stats['instance_changes'],
    name='Instance Changes',
    orientation='h',
    marker=dict(color='#E74C3C', line=dict(color='white', width=1)),
    hovertemplate='<b>%{y}</b><br>' +
                  'Instance Changes: %{x}<br>' +
                  '<extra></extra>'
))

fig.update_layout(
    barmode='group',
    title={
        'text': '👤 Owner-Based Cluster Analysis',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 24, 'color': '#2C3E50', 'family': 'Arial Black'}
    },
    xaxis={
        'title': 'Count',
        'showgrid': True,
        'gridcolor': '#E8E8E8',
        'tickfont': {'size': 12}
    },
    yaxis={
        'title': 'Cluster Owner',
        'tickfont': {'size': 11, 'family': 'Arial'},
        'showgrid': False
    },
    height=700,
    plot_bgcolor='white',
    paper_bgcolor='#F8F9FA',
    hovermode='closest',
    font={'family': 'Arial, sans-serif', 'size': 12, 'color': '#2C3E50'},
    margin=dict(l=250, r=50, t=80, b=50),
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1,
        font=dict(size=12, family='Arial')
    )
)

displayHTML(pyo.plot(fig, output_type='div', include_plotlyjs='cdn'))

In [0]:
import plotly.graph_objs as go
import plotly.offline as pyo

# Query workspace and cluster details
df = spark.sql("""
    SELECT 
      workspace_name,
      cluster_name,
      change_impact,
      before_driver_instance,
      after_driver_instance,
      before_worker_instance,
      after_worker_instance,
      total_changes_count
    FROM dev_sandbox.billing_forecast.cluster_config_backup
    WHERE update_status = 'SUCCESS'
    ORDER BY workspace_name, cluster_name
""")

pdf = df.toPandas()

# Create sunburst chart showing workspace -> cluster hierarchy
fig = go.Figure(go.Sunburst(
    labels=(['All Workspaces'] + 
            pdf['workspace_name'].tolist() + 
            pdf['cluster_name'].tolist()),
    parents=([''] + 
             ['All Workspaces'] * len(pdf) + 
             pdf['workspace_name'].tolist()),
    values=([pdf['total_changes_count'].sum()] + 
            pdf['total_changes_count'].tolist() + 
            pdf['total_changes_count'].tolist()),
    branchvalues='total',
    marker=dict(
        colorscale='RdYlGn_r',
        cmid=pdf['total_changes_count'].mean(),
        line=dict(color='white', width=2)
    ),
    hovertemplate='<b>%{label}</b><br>' +
                  'Changes: %{value}<br>' +
                  '<extra></extra>'
))

fig.update_layout(
    title={
        'text': '🏢 Workspace → Cluster Hierarchy',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 24, 'color': '#2C3E50', 'family': 'Arial Black'}
    },
    height=700,
    paper_bgcolor='#F8F9FA',
    font={'family': 'Arial, sans-serif', 'size': 12, 'color': '#2C3E50'}
)

displayHTML(pyo.plot(fig, output_type='div', include_plotlyjs='cdn'))

In [0]:
import plotly.graph_objs as go
import plotly.offline as pyo
import pandas as pd

# Query cluster instance type changes
df = spark.sql("""
    SELECT 
      cluster_name,
      workspace_name,
      before_driver_instance,
      after_driver_instance,
      before_worker_instance,
      after_worker_instance,
      change_impact
    FROM dev_sandbox.billing_forecast.cluster_config_backup
    WHERE update_status = 'SUCCESS'
      AND instance_type_changed = true
    ORDER BY cluster_name
""")

pdf = df.toPandas()

# Extract owner
pdf['owner'] = pdf['cluster_name'].apply(lambda x: x.split("'s")[0] if "'s" in x else x.split()[0] if len(x.split()) > 0 else 'Unknown')

# Create a detailed comparison chart
fig = go.Figure()

# Add before driver instances
fig.add_trace(go.Bar(
    name='Before (Driver)',
    x=pdf['cluster_name'],
    y=[1] * len(pdf),
    text=pdf['before_driver_instance'],
    textposition='inside',
    marker=dict(color='#FF6B6B', line=dict(color='white', width=1)),
    hovertemplate='<b>%{x}</b><br>' +
                  'Before Driver: %{text}<br>' +
                  '<extra></extra>',
    customdata=pdf[['before_driver_instance']].values
))

# Add after driver instances
fig.add_trace(go.Bar(
    name='After (Driver)',
    x=pdf['cluster_name'],
    y=[1] * len(pdf),
    text=pdf['after_driver_instance'],
    textposition='inside',
    marker=dict(color='#4ECDC4', line=dict(color='white', width=1)),
    hovertemplate='<b>%{x}</b><br>' +
                  'After Driver: %{text}<br>' +
                  '<extra></extra>',
    customdata=pdf[['after_driver_instance']].values
))

# Add before worker instances
fig.add_trace(go.Bar(
    name='Before (Worker)',
    x=pdf['cluster_name'],
    y=[1] * len(pdf),
    text=pdf['before_worker_instance'],
    textposition='inside',
    marker=dict(color='#FFE66D', line=dict(color='white', width=1)),
    hovertemplate='<b>%{x}</b><br>' +
                  'Before Worker: %{text}<br>' +
                  '<extra></extra>',
    customdata=pdf[['before_worker_instance']].values
))

# Add after worker instances
fig.add_trace(go.Bar(
    name='After (Worker)',
    x=pdf['cluster_name'],
    y=[1] * len(pdf),
    text=pdf['after_worker_instance'],
    textposition='inside',
    marker=dict(color='#95E1D3', line=dict(color='white', width=1)),
    hovertemplate='<b>%{x}</b><br>' +
                  'After Worker: %{text}<br>' +
                  '<extra></extra>',
    customdata=pdf[['after_worker_instance']].values
))

fig.update_layout(
    barmode='stack',
    title={
        'text': '🔄 Cluster Instance Type Transitions',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 24, 'color': '#2C3E50', 'family': 'Arial Black'}
    },
    xaxis={
        'title': 'Cluster Name',
        'tickangle': -45,
        'tickfont': {'size': 9},
        'showgrid': False
    },
    yaxis={
        'title': 'Instance Configuration',
        'showgrid': False,
        'showticklabels': False
    },
    height=600,
    plot_bgcolor='white',
    paper_bgcolor='#F8F9FA',
    hovermode='closest',
    font={'family': 'Arial, sans-serif', 'size': 12, 'color': '#2C3E50'},
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1,
        font=dict(size=12, family='Arial')
    )
)

displayHTML(pyo.plot(fig, output_type='div', include_plotlyjs='cdn'))

In [0]:
import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.express as px

# Query detailed cluster and owner information
df = spark.sql("""
    SELECT 
      cluster_name,
      workspace_name,
      before_driver_instance,
      after_driver_instance,
      before_worker_instance,
      after_worker_instance,
      change_impact,
      total_changes_count,
      instance_type_changed,
      policy_changed,
      spark_config_changed,
      tags_changed,
      autoscale_changed
    FROM dev_sandbox.billing_forecast.cluster_config_backup
    WHERE update_status = 'SUCCESS'
""")

pdf = df.toPandas()

# Extract owner
pdf['owner'] = pdf['cluster_name'].apply(lambda x: x.split("'s")[0] if "'s" in x else x.split()[0] if len(x.split()) > 0 else 'Unknown')

# Create a scatter plot showing clusters by owner and change impact
fig = px.scatter(
    pdf,
    x='owner',
    y='total_changes_count',
    size='total_changes_count',
    color='change_impact',
    hover_name='cluster_name',
    hover_data={
        'owner': True,
        'workspace_name': True,
        'before_driver_instance': True,
        'after_driver_instance': True,
        'total_changes_count': True
    },
    color_discrete_map={'MAJOR': '#E74C3C', 'MODERATE': '#F39C12', 'MINOR': '#27AE60'},
    size_max=50
)

fig.update_layout(
    title={
        'text': '🎯 Owner vs Cluster Changes Scatter',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 24, 'color': '#2C3E50', 'family': 'Arial Black'}
    },
    xaxis={
        'title': 'Cluster Owner',
        'tickangle': -45,
        'tickfont': {'size': 10},
        'showgrid': True,
        'gridcolor': '#E8E8E8'
    },
    yaxis={
        'title': 'Number of Changes',
        'showgrid': True,
        'gridcolor': '#E8E8E8'
    },
    height=600,
    plot_bgcolor='white',
    paper_bgcolor='#F8F9FA',
    hovermode='closest',
    font={'family': 'Arial, sans-serif', 'size': 12, 'color': '#2C3E50'},
    legend=dict(
        title={'text': 'Change Impact', 'font': {'size': 14}},
        orientation='v',
        yanchor='top',
        y=1,
        xanchor='left',
        x=1.02,
        font=dict(size=12, family='Arial')
    )
)

displayHTML(pyo.plot(fig, output_type='div', include_plotlyjs='cdn'))

In [0]:
import plotly.graph_objs as go
import plotly.offline as pyo

# Query detailed cluster comparison data
df = spark.sql("""
    SELECT 
      cluster_name,
      workspace_name,
      before_driver_instance,
      after_driver_instance,
      before_worker_instance,
      after_worker_instance,
      before_autoscale_min,
      after_autoscale_min,
      before_autoscale_max,
      after_autoscale_max,
      change_impact,
      change_summary,
      DATE(backup_timestamp) as backup_date
    FROM dev_sandbox.billing_forecast.cluster_config_backup
    WHERE update_status = 'SUCCESS'
    ORDER BY cluster_name
""")

pdf = df.toPandas()

# Extract owner
pdf['owner'] = pdf['cluster_name'].apply(lambda x: x.split("'s")[0] if "'s" in x else x.split()[0] if len(x.split()) > 0 else 'Unknown')

# Create color coding based on change impact
fill_colors = []
for impact in pdf['change_impact']:
    if impact == 'MAJOR':
        fill_colors.append('rgba(231, 76, 60, 0.15)')
    elif impact == 'MODERATE':
        fill_colors.append('rgba(243, 156, 18, 0.15)')
    else:
        fill_colors.append('rgba(39, 174, 96, 0.15)')

# Select key columns for display
display_cols = ['owner', 'cluster_name', 'workspace_name', 'change_impact', 
                'before_driver_instance', 'after_driver_instance',
                'before_worker_instance', 'after_worker_instance',
                'change_summary']

fig = go.Figure(
    data=[go.Table(
        header=dict(
            values=['<b>' + col.replace('_', ' ').title() + '</b>' for col in display_cols],
            fill_color='#34495E',
            align='left',
            font=dict(color='white', size=12, family='Arial Black'),
            height=40
        ),
        cells=dict(
            values=[pdf[col] for col in display_cols],
            fill_color=[fill_colors] * len(display_cols),
            align='left',
            font=dict(color='#2C3E50', size=10, family='Arial'),
            height=35,
            line=dict(color='white', width=1)
        )
    )]
)

fig.update_layout(
    title={
        'text': '📋 Detailed Cluster Comparison by Owner',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 24, 'color': '#2C3E50', 'family': 'Arial Black'}
    },
    height=800,
    paper_bgcolor='#F8F9FA',
    font={'family': 'Arial, sans-serif', 'size': 12, 'color': '#2C3E50'}
)

displayHTML(pyo.plot(fig, output_type='div', include_plotlyjs='cdn'))

In [0]:
import plotly.graph_objs as go
import plotly.offline as pyo

# Query owner statistics
df = spark.sql("""
    SELECT 
      cluster_name,
      workspace_name,
      change_impact,
      total_changes_count,
      instance_type_changed,
      policy_changed,
      spark_config_changed,
      tags_changed,
      init_scripts_changed,
      autotermination_changed,
      runtime_engine_changed,
      security_mode_changed,
      autoscale_changed
    FROM dev_sandbox.billing_forecast.cluster_config_backup
    WHERE update_status = 'SUCCESS'
""")

pdf = df.toPandas()

# Extract owner
pdf['owner'] = pdf['cluster_name'].apply(lambda x: x.split("'s")[0] if "'s" in x else x.split()[0] if len(x.split()) > 0 else 'Unknown')

# Aggregate change types by owner
change_types = ['instance_type_changed', 'policy_changed', 'spark_config_changed', 
                'tags_changed', 'init_scripts_changed', 'autotermination_changed',
                'runtime_engine_changed', 'security_mode_changed', 'autoscale_changed']

owner_changes = pdf.groupby('owner')[change_types].sum().reset_index()

# Create stacked bar chart
fig = go.Figure()

colors = ['#E74C3C', '#3498DB', '#2ECC71', '#F39C12', '#9B59B6', '#1ABC9C', '#E67E22', '#95A5A6', '#34495E']

for i, change_type in enumerate(change_types):
    fig.add_trace(go.Bar(
        name=change_type.replace('_', ' ').title().replace(' Changed', ''),
        x=owner_changes['owner'],
        y=owner_changes[change_type],
        marker_color=colors[i],
        hovertemplate='<b>%{x}</b><br>' +
                      change_type.replace('_', ' ').title() + ': %{y}<br>' +
                      '<extra></extra>'
    ))

fig.update_layout(
    barmode='stack',
    title={
        'text': '📊 Change Types by Owner',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 24, 'color': '#2C3E50', 'family': 'Arial Black'}
    },
    xaxis={
        'title': 'Cluster Owner',
        'tickangle': -45,
        'tickfont': {'size': 11},
        'showgrid': False
    },
    yaxis={
        'title': 'Number of Changes',
        'showgrid': True,
        'gridcolor': '#E8E8E8'
    },
    height=600,
    plot_bgcolor='white',
    paper_bgcolor='#F8F9FA',
    hovermode='closest',
    font={'family': 'Arial, sans-serif', 'size': 12, 'color': '#2C3E50'},
    legend=dict(
        orientation='v',
        yanchor='top',
        y=1,
        xanchor='left',
        x=1.02,
        font=dict(size=10, family='Arial')
    )
)

displayHTML(pyo.plot(fig, output_type='div', include_plotlyjs='cdn'))

In [0]:
import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.express as px

# Query cluster and workspace summary
df = spark.sql("""
    SELECT 
      cluster_name,
      workspace_name,
      change_impact,
      total_changes_count,
      before_driver_instance,
      after_driver_instance,
      before_worker_instance,
      after_worker_instance
    FROM dev_sandbox.billing_forecast.cluster_config_backup
    WHERE update_status = 'SUCCESS'
""")

pdf = df.toPandas()

# Extract owner
pdf['owner'] = pdf['cluster_name'].apply(lambda x: x.split("'s")[0] if "'s" in x else x.split()[0] if len(x.split()) > 0 else 'Unknown')

# Create treemap showing workspace -> owner -> cluster hierarchy
fig = px.treemap(
    pdf,
    path=['workspace_name', 'owner', 'cluster_name'],
    values='total_changes_count',
    color='change_impact',
    color_discrete_map={'MAJOR': '#E74C3C', 'MODERATE': '#F39C12', 'MINOR': '#27AE60'},
    hover_data={
        'before_driver_instance': True,
        'after_driver_instance': True,
        'before_worker_instance': True,
        'after_worker_instance': True,
        'total_changes_count': True
    }
)

fig.update_layout(
    title={
        'text': '🌳 Workspace → Owner → Cluster Treemap',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 24, 'color': '#2C3E50', 'family': 'Arial Black'}
    },
    height=700,
    paper_bgcolor='#F8F9FA',
    font={'family': 'Arial, sans-serif', 'size': 12, 'color': '#2C3E50'}
)

fig.update_traces(
    textinfo='label+value',
    textfont=dict(size=12, family='Arial Black'),
    marker=dict(line=dict(color='white', width=2))
)

displayHTML(pyo.plot(fig, output_type='div', include_plotlyjs='cdn'))